# Orbitrap Astral DIA (OADIA) - MS2 Windows

In [ ]:
# import public packages
import pandas as pd
import os

# !pip install pydiaid
import pydiaid.oadia.method_generator as method_creator
import pydiaid.oadia.method_evaluation as method_evaluator

from IPython.display import Image

# # importing components for visualization
%load_ext autoreload
%autoreload 2

## Load a proteomics library

In [ ]:
import pydiaid.loader as loader

library_name = "/Users/patriciaskowronek/Documents/test_lib_py_diaid/DIANN_5_6min_50ng_syP_2equiScans_wide_lib.tsv"
analysis_software = 'DIANN single-run'
ptm = "None" #Options: 'None', 'Phospho', 'DiGly', ["STY"]

In [ ]:
library = loader.load_library(
        library_name,
        analysis_software,
        ptm,
        False
    )
library

## Define where the output should be saved

In [ ]:
import pydiaid.synchropasef.method_creator as syP_method_creator
folder_path = "/Users/patriciaskowronek/Documents/test_folder_2"
syP_method_creator.create_folder([folder_path])

## Create histogram of precursors in m/z

In [ ]:
import pydiaid.synchropasef.method_evaluator as syP_method_evaluator
import pydiaid.synchropasef.plots as syP_plots

plot_parameters = {
    "plot_mz": [100, 1400],
}

syP_plots.plot_precursor_distribution_as_histogram(
        library,
        plot_parameters,
        os.path.join(
            folder_path,
            'Histogram_precursor_distribution_in_library.png'
        ),
        gui=True
    ),

dict_charge_of_precursor = syP_method_evaluator.calculate_percentage_multiple_charged_precursors(library)
mult_charged_precursor_info = pd.DataFrame(
    {
        "charge state of precursors": list(dict_charge_of_precursor.keys()),
        "ratio [%]": list(dict_charge_of_precursor.values())
    }
)
mult_charged_precursor_info.to_csv(
    os.path.join(
        folder_path,
        'percentage_of_multiple_charged_precursors.csv'
    ),
    index=False
)
print(mult_charged_precursor_info)

## Specify method parameters

In [ ]:
mz_values=library["mz"]
mz_range=(300, 1200)
num_bins=20
window_type = "variable" # fixed or variable
# window_type = "fixed"
output_format = "all" #targeted, center_mass, mz_ranges

min_width=1.0
max_width=40.0

method_evaluator.calculate_precursor_within_scan_area(
    library,
    mz_range,
)

## Generate method

In [ ]:
df_window, bins = method_creator.create_method(
    mz_values,
    mz_range,
    num_bins,
    window_type,
    output_format,
    folder_path
)

## Load and Evaluate method

In [ ]:
df_window, bins, df_method = method_evaluator.load_method_file(f"{folder_path}/OA_DIA_method_{mz_range[0]}_{mz_range[1]}_{num_bins}_{window_type}"+"_center_mass.csv")
df_method

In [ ]:
# Get statistics with detailed analysis
stats = method_evaluator.analyze_bins(bins, mz_values)

# Print summary
print("Summary Statistics:")
print(f"Number of bins: {stats['num_bins']}")
print(f"Min width: {stats['min_width']:.2f}")
print(f"Max width: {stats['max_width']:.2f}")
print(f"Average width: {stats['avg_width']:.2f}")
print(f"Min items per bin: {stats['min_count']}")
print(f"Max items per bin: {stats['max_count']}")
print(f"Average items per bin: {stats['avg_count']:.2f}")

if round(stats['min_width'],2) == round(stats['max_width'], 2):
    window_type = "fixed"
else:
    window_type = "variable"

# Print detailed tabular view
print("\n" + stats['tabular_stats'])

In [ ]:
df_window = method_creator.calculate_window_metrics(bins)
method_evaluator.plot_precursors_per_scan(
    mz_values, 
    bins, 
    window_type,
    df_window, 
    os.path.join(
        folder_path,
        'precursor_per_scan.png'
    ),
    gui=True
)
method_evaluator.plot_precursors_per_scan(
    mz_values, 
    bins, 
    window_type,
    df_window, 
    os.path.join(
        folder_path,
        'precursor_per_scan.png'
    ),
)